 <h1>SpaceX Falcon 9 first stage Landing Prediction</h1>

<h2>Lab 6: Building an interactive dashboard using Plotly and Dash</h2>

In [2]:
!pip install pandas dash
!pip install wget

  Using cached dash-2.3.1-py3-none-any.whl (9.6 MB)
  Using cached dash_table-5.0.0-py3-none-any.whl (3.9 kB)
  Using cached dash_core_components-2.0.0-py3-none-any.whl (3.8 kB)
  Using cached dash_html_components-2.0.0-py3-none-any.whl (4.1 kB)


In [8]:
!python -m wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv


Saved under spacex_launch_dash.csv


In [10]:
import pandas as pd
spacex_data = pd.read_csv('spacex_launch_dash.csv')
spacex_data.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [13]:
### Install ngrok
!python -m wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!python -m unzip ngrok-stable-linux-amd64.zip

### Run ngrok to tunnel Dash app port 8050 to the outside world. 
### This command runs in the background.
get_ipython().system_raw('./ngrok http 8050 &')

### Get the public URL where you can access the Dash app. Copy this URL.
!python -m curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


Saved under ngrok-stable-linux-amd64 (1).zip


C:\Users\DELL\anaconda3\python.exe: No module named unzip
C:\Users\DELL\anaconda3\python.exe: No module named curl.__main__; 'curl' is a package and cannot be directly executed
Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Manage App Execution Aliases.


In [14]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                html.Label('Dropdown'),
                                dcc.Dropdown(id='site-dropdown',
                                    options=[
                                             {'label':'All Sites', 'value':'All Sites'},
                                             {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                             {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                             {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
                                             {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'}
                                    ],
                                    value='All Sites',
                                    placeholder='Select a Launch Site here',searchable=True
                                ),
                                html.Br(),

                                # Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000,
                                                marks={i: '{} (Kg)'.format(i) for i in range(0, 10001, 1000)},
                                                value=[min_payload, max_payload]),

                                # Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'))
def get_pie(site_sel):
  if site_sel == 'All Sites':
    piedata = spacex_df[['Launch Site', 'class']]
    piedat = piedata.groupby(['Launch Site']).sum()
    pie_fig = px.pie(piedat, values="class", names=piedat.index, title='Total successful launches by site')
    return pie_fig
  else:
    piedata = spacex_df[spacex_df['Launch Site']==site_sel]
    piedat = piedata['class'].value_counts()
    pie_fig = px.pie(piedat, names=piedat.index, values=piedat.values, title='Total successful launches for site {}'.format(site_sel))
    return pie_fig

# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback( Output(component_id='success-payload-scatter-chart', component_property='figure'),
               [Input(component_id='site-dropdown', component_property='value'),
                Input(component_id='payload-slider', component_property='value')])
def get_sct(sel_site, payload):
  if sel_site == 'All Sites':
    sctdata = spacex_df[['Launch Site', 'class', 'Payload Mass (kg)', 'Booster Version Category']]
    sctdat = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload[0]) & (spacex_df['Payload Mass (kg)'] <= payload[1])]
    sct_fig = px.scatter(sctdat, x="Payload Mass (kg)", y="class", color="Booster Version Category")
    return sct_fig
  else:
    sctdata = spacex_df[['Launch Site', 'class', 'Payload Mass (kg)', 'Booster Version Category']]
    sctdat1 = sctdata[sctdata['Launch Site']==sel_site]
    sctdat = sctdat1[(sctdat1['Payload Mass (kg)'] >= payload[0]) & (sctdat1['Payload Mass (kg)'] <= payload[1])]
    sct_fig = px.scatter(sctdat, x="Payload Mass (kg)", y="class", color="Booster Version Category")
    return sct_fig
# Run the app
if __name__ == '__main__':
    app.run_server()

C:\Users\DELL\AppData\Local\Temp/ipykernel_11952/2416008115.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\DELL\AppData\Local\Temp/ipykernel_11952/2416008115.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [11/May/2022 19:03:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2022 19:03:51] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_3_1m1652275622.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2022 19:03:51] "GET /_dash-component-suites/dash/deps/react@16.v2_3_1m1652275622.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2022 19:03:51] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_3_1m1652275622.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2022 19:03:51] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_3_1m1652275622.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2022 19:03:51] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_3_1m1652275621.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2022 19:03:51] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_3_0m1652275622.js HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2022 19:03:51] "GET /_dash-component-s